In [1]:
from os import access
import xlwings as xw
from connect_to_dhan import Connection
from dhanhq import dhanhq
import numpy as np
import pandas as pd
from dhanhq import marketfeed
import yaml
import time



In [2]:
k = [None for k in range(11)]

In [ ]:
)

11

In [2]:

""" configuring the workbook and getting the security info
"""
# Load Excel file
excel_file = 'DhanTrading.xlsx'
workbook = xw.Book(excel_file)
TradeSheet = workbook.sheets['Trade']
OptionsLookUp = workbook.sheets['OptionsLookUp'] 
IndexLookup = workbook.sheets['IndexLookup']

In [3]:
def option_age(x):  # This function will classify the options strike if it belongs to Current series, Next series or Far next.
    if x == 1.0:
        return 'C'  # Current series
    elif x == 2.0:
        return 'N'  # Next Series
    else:
        return 'F'  # Far next series

def connect_to_dhan():
    config_file_path = r".\\config\\config.yaml" #This has the API key and client id
    with open(config_file_path,'r') as config:
        api_config = yaml.safe_load(config)
    clinet_id = api_config.get('api_config')[1]
    access_token = api_config.get('api_config')[2]

    # Establish connection to Dhan
    try:
        DhanConnector = Connection(clinet_id, access_token)
        ConnectionObject = DhanConnector.connect_dhan()
        dhan = ConnectionObject['conn']
    except Exception as e :
        raise ConnectionError(f"Can't connect {e}")  

    return({"connection":dhan,
            "client_id":clinet_id,
            "access_token":access_token})    

Configure the sheets

In [27]:
def initial_sheet_config():
    connections__= connect_to_dhan() #returned as dictionary but accessed like a list
    dhan = connections__['connection']
    client_id = connections__['client_id']
    access_token = connections__['access_token']
    # Load the latest keys and scripts metadata automatically
    security_list = dhan.fetch_security_list("compact")
    #--------------------------------------------------------------------------------------------
    # index
    #--------------------------------------------------------------------------------------------
    mcx_fut =  security_list[(security_list['SEM_EXM_EXCH_ID']=='MCX') & 
                                (security_list['SM_SYMBOL_NAME']=='CRUDEOIL') & 
                                (security_list['SEM_INSTRUMENT_NAME']=='FUTCOM')]
    index = security_list[(security_list['SEM_INSTRUMENT_NAME'] == 'INDEX') & (
        (security_list['SEM_TRADING_SYMBOL']=='BANKNIFTY') |
        (security_list['SEM_TRADING_SYMBOL']=='NIFTY')) & (security_list['SEM_SEGMENT']=='I')]
    filtered_df_index = pd.concat([mcx_fut,index])
    filtered_df_index = filtered_df_index.copy()
    filtered_df_index['SEM_TRADING_SYMBOL']= [x[0] for x in filtered_df_index.SEM_TRADING_SYMBOL.str.split('-')]

    filtered_df_index['Rank'] = filtered_df_index.groupby('SEM_EXM_EXCH_ID')['SEM_EXPIRY_DATE'].rank(method='dense', ascending=True)
    filtered_df_index = filtered_df_index[(filtered_df_index['Rank']==1.0) | (np.isnan(filtered_df_index['Rank']))]
    IndexLookup.clear()
    IndexLookup.range('A1').options(index=False).value = filtered_df_index[['SEM_TRADING_SYMBOL','SEM_SMST_SECURITY_ID']]
    #--------------------------------------------------------------------------------------------
    # Filter for only NIFTY and BANKNIFTY OPTIONS and MCX
    #--------------------------------------------------------------------------------------------
    NSE = security_list[(
        (security_list['SEM_EXM_EXCH_ID'] == 'NSE') & 
        (security_list['SEM_INSTRUMENT_NAME'] == 'OPTIDX') & 
        ((security_list['SEM_TRADING_SYMBOL'].str.startswith('BANKNIFTY')) | 
            (security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTY')))
        &
        (security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTYNXT50') == False)
    )]    
    MCX = security_list[(security_list['SEM_EXM_EXCH_ID'] == 'MCX') & 
                        (security_list['SEM_INSTRUMENT_NAME'] == 'OPTFUT') & 
                        (security_list['SM_SYMBOL_NAME'] == 'CRUDEOIL')]

    filtered_df = pd.concat([NSE, MCX])
    filtered_df = filtered_df.copy()  

    # Below logic will help us to classify the Series type {C - Current, N - Next and F - Future}
    filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = np.nan
    filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = pd.to_datetime(filtered_df.SEM_EXPIRY_DATE)
    filtered_df['SEM_UNDERLYING'] = [x[0] for x in filtered_df.SEM_CUSTOM_SYMBOL.str.split(' ')]
    filtered_df['Rank'] = filtered_df.groupby('SEM_UNDERLYING')['SEM_EXPIRY_DATE_CUSTOM'].rank(method='dense', ascending=True)
    filtered_df['Series'] = filtered_df['Rank'].apply(option_age)

    filtered_df = filtered_df[filtered_df['Series'] == 'C'].copy(deep=True)
    filtered_df['SEM_STRIKE_PRICE'] = filtered_df['SEM_STRIKE_PRICE'].astype(int)
    filtered_df = filtered_df[['SEM_SMST_SECURITY_ID','SEM_UNDERLYING','SEM_OPTION_TYPE','SEM_STRIKE_PRICE']].copy(deep=True)
    filtered_df['Series'] = filtered_df['SEM_UNDERLYING']+"_"+filtered_df['SEM_OPTION_TYPE']+"_"+filtered_df['SEM_STRIKE_PRICE'].astype(str)
    OptionsLookUp.clear()
    OptionsLookUp.range('A1').options(index=False).value = filtered_df[['Series','SEM_SMST_SECURITY_ID']]

In [31]:
def refresh_instruments():
    index_key = str(int(TradeSheet.range("IndexKey").value))
    atm_key = str(int(TradeSheet.range("ATM_KEY").value))
    itm_1_key = str(int(TradeSheet.range("ITM_ONE_KEY").value))
    itm_2_key = str(int(TradeSheet.range("ITM_TWO_KEY").value))
    itm_3_key = str(int(TradeSheet.range("ITM_THREE_KEY").value))
    range_mapping = {
        index_key : 'INDEX_LTP',
        atm_key : 'ATM_LTP',
        itm_1_key : 'ITM_ONE_LTP',
        itm_2_key : 'ITM_TWO_LTP',
        itm_3_key : 'ITM_THREE_LTP'
    }
    return(range_mapping)

In [33]:
inst = refresh_instruments()

In [51]:
dhanhq.ohlc_data(securities={"NSE_FNO":[13]})
    

TypeError: dhanhq.ohlc_data() missing 1 required positional argument: 'self'

In [46]:
list(inst.keys())[1:]

['441307', '441318', '441320', '441322']

In [55]:
connections__= connect_to_dhan() #returned as dictionary but accessed like a list
dhan = connections__['connection']

In [61]:
k = dhan.ohlc_data(securities={"IDX_I":[13]})

In [60]:
dhan.ohlc_data(securities={"MCX_COMM":[436953]})

{'status': 'success',
 'remarks': '',
 'data': {'data': {'MCX_COMM': {'436953': {'last_price': 5914,
     'ohlc': {'open': 5944, 'close': 5938, 'high': 5976, 'low': 5891}}}},
  'status': 'success'}}

In [4]:
    connections__= connect_to_dhan() #returned as dictionary but accessed like a list
    dhan = connections__['connection']
    client_id = connections__['client_id']
    access_token = connections__['access_token']

In [67]:
security_to_cell = {'436953': 'INDEX_LTP', '441317': 'ATM_LTP', '441318': 'ITM_ONE_LTP', '441319': 'ITM_TWO_LTP', '441320': 'ITM_THREE_LTP'}

In [68]:
dhan.place_order('41690')

In [5]:
SLICE_ORDER_BUY = dhan.place_order(security_id='41690', exchange_segment=dhanhq.NSE_FNO, transaction_type=dhanhq.BUY, quantity=25,
                           order_type=dhanhq.LIMIT, product_type=dhanhq.INTRA, price=80, trigger_price= 80-0.1, disclosed_quantity=0,
                           after_market_order=False, validity='DAY', amo_time='OPEN',
                           bo_profit_value=None, bo_stop_loss_Value=None, tag="ALGOENTRY") 

In [8]:
dhan.cancel_order( SLICE_ORDER_BUY['data']['orderId'])

{'status': 'success',
 'remarks': '',
 'data': {'orderId': '42241224508416', 'orderStatus': 'CANCELLED'}}

In [69]:
if security_id in security_to_cell:
    cell_reference = security_to_cell[security_id]
    print(cell_reference)
    #TradeSheet.range(cell_reference).value = response['LTP']

ATM_LTP


In [70]:
TradeSheet.range(cell_reference).value

145.0

In [71]:
security_to_cell['436953']

'INDEX_LTP'

In [21]:
dhan.get_order_by_correlationID("ALGOLOSS")

{'status': 'success',
 'remarks': '',
 'data': {'dhanClientId': '1100381471',
  'orderId': '102241224619816',
  'exchangeOrderId': '1200000133301412',
  'correlationId': 'ALGOLOSS',
  'orderStatus': 'TRADED',
  'transactionType': 'SELL',
  'exchangeSegment': 'NSE_FNO',
  'productType': 'INTRADAY',
  'orderType': 'LIMIT',
  'validity': 'DAY',
  'tradingSymbol': 'NIFTY-Dec2024-23800-CE',
  'securityId': '41687',
  'quantity': 25,
  'disclosedQuantity': 0,
  'price': 78.9,
  'triggerPrice': 0.0,
  'afterMarketOrder': False,
  'boProfitValue': 0.0,
  'boStopLossValue': 0.0,
  'legName': 'NA',
  'createTime': '2024-12-24 12:54:07',
  'updateTime': '2024-12-24 12:54:09',
  'exchangeTime': '2024-12-24 12:54:08',
  'drvExpiryDate': '2024-12-26',
  'drvOptionType': 'CALL',
  'drvStrikePrice': 23800.0,
  'omsErrorCode': '0',
  'omsErrorDescription': 'TRADE CONFIRMED',
  'algoId': '0',
  'remainingQuantity': 0,
  'averageTradedPrice': 79.15,
  'filledQty': 25}}

In [14]:
algo_entry = pd.DataFrame(dhan.get_order_by_correlationID("ALGOENTRY")['data'])

ValueError: If using all scalar values, you must pass an index

In [22]:
pd.DataFrame(dhan.get_trade_book()['data'])

,dhanClientId,orderId,exchangeOrderId,exchangeTradeId,transactionType,exchangeSegment,productType,orderType,tradingSymbol,customSymbol,securityId,tradedQuantity,tradedPrice,createTime,updateTime,exchangeTime,drvExpiryDate,drvOptionType,drvStrikePrice
0,1100381471,10124122424093,1200000024877188,402814470,BUY,NSE_EQ,CNC,MARKET,OLAELEC,None,24777,3,94.22,2024-12-24 11:11:23,2024-12-24 11:11:23,2024-12-24 11:11:23,0001-01-01,NA,0.00
1,1100381471,10124122424093,1200000024877188,402814471,BUY,NSE_EQ,CNC,MARKET,OLAELEC,None,24777,8,94.23,2024-12-24 11:11:23,2024-12-24 11:11:23,2024-12-24 11:11:23,0001-01-01,NA,0.00
2,1100381471,1224122462316,1100000009423060,429155,SELL,NSE_FNO,INTRADAY,LIMIT,NIFTY-Dec2024-23750-PE,None,41686,50,97.80,2024-12-24 09:32:52,2024-12-24 09:32:52,2024-12-24 09:32:52,2024-12-26,NA,23750.00
3,1100381471,1224122462316,1100000009423060,429156,SELL,NSE_FNO,INTRADAY,LIMIT,NIFTY-Dec2024-23750-PE,None,41686,50,97.80,2024-12-24 09:32:52,2024-12-24 09:32:52,2024-12-24 09:32:52,2024-12-26,NA,23750.00
4,1100381471,2224122454316,1400000009586327,335764,BUY,NSE_FNO,INTRADAY,MARKET,NIFTY-Dec2024-23600-CE,None,41672,25,206.45,2024-12-24 09:30:09,2024-12-24 09:30:09,2024-12-24 09:30:09,2024-12-26,NA,23600.00
5,1100381471,2224122459716,1400000010670199,367383,SELL,NSE_FNO,INTRADAY,MARKET,NIFTY-Dec2024-23600-CE,None,41672,25,193.95,2024-12-24 09:31:50,2024-12-24 09:31:50,2024-12-24 09:31:50,2024-12-26,NA,23600.00
6,1100381471,4224122473016,1100000009269255,425266,BUY,NSE_FNO,INTRADAY,LIMIT,NIFTY-Dec2024-23750-PE,None,41686,50,93.10,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-26,NA,23750.00
7,1100381471,4224122473016,1100000009269255,425272,BUY,NSE_FNO,INTRADAY,LIMIT,NIFTY-Dec2024-23750-PE,None,41686,25,93.10,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-26,NA,23750.00
8,1100381471,4224122473016,1100000009269255,425273,BUY,NSE_FNO,INTRADAY,LIMIT,NIFTY-Dec2024-23750-PE,None,41686,25,93.10,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-24 09:32:46,2024-12-26,NA,23750.00
9,1100381471,12241224174116,1300000027523856,649684,SELL,NSE_FNO,INTRADAY,MARKET,NIFTY-Dec2024-23850-PE,None,41690,25,102.35,2024-12-24 10:06:24,2024-12-24 10:06:24,2024-12-24 10:06:24,2024-12-26,NA,23850.00
